In [ ]:
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import xlwings as xw
import threading
import time

book = xw.Book('Richspread.xlsx') 
sht = book.sheets['Placing_Stock']
        

class PlaceOrderApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId):
        
        self.place_my_order(orderId)

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice,
                    permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"OrderStatus. ID: {orderId}, Status: {status}, Filled: {filled}, Remaining: {remaining}, AvgFillPrice: {avgFillPrice}")

    def openOrder(self, orderId, contract, order, orderState):
        print(f"OpenOrder. ID: {orderId}, Symbol: {contract.symbol}, Action: {order.action}, Quantity: {order.totalQuantity}")
        
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson=None):
        """Handle errors and informational messages."""
        
        if errorCode in [2104, 2106, 2158]:
            return  
        print(f"Error. ID: {reqId}, Code: {errorCode}, Msg: {errorString}")
        if advancedOrderRejectJson:
            print(f"Advanced Reject: {advancedOrderRejectJson}")

    def execDetails(self, reqId, contract, execution):
        print(f"ExecDetails. Symbol: {contract.symbol}, ExecId: {execution.execId}, Shares: {execution.shares}")

    def place_my_order(self, orderId):
        
        contract = Contract()
        contract.symbol = sht.range('B2').value
        contract.secType = sht.range('B3').value
        contract.exchange = sht.range('B4').value
        contract.currency = sht.range('B5').value

        # Market order
        order = Order()
        order.action = sht.range('B9').value
        order.orderType = sht.range('B10').value
        order.totalQuantity = sht.range('B11').value
        order.lmtPrice = sht.range('B12').value
        order.sweepToFill = True
        

        self.placeOrder(orderId, contract, order)
        print(f"Placed MARKET order for {order.totalQuantity} shares of {contract.symbol}")

        # Stop order
        stop_order = Order()
        stop_order.action = sht.range('B17').value
        stop_order.orderType = sht.range('B18').value
        stop_order.totalQuantity = sht.range('B19').value
        stop_order.auxPrice = sht.range('B20').value

        self.placeOrder(orderId + 1, contract, stop_order)
        print(f"Placed STOP order for {stop_order.totalQuantity} shares of {contract.symbol} at ${stop_order.auxPrice}")

def main():
    app = PlaceOrderApp()
    app.connect("127.0.0.1", 7497, 123)
    thread = threading.Thread(target=app.run, daemon=True)
    thread.start()
    time.sleep(10)  
    app.disconnect()

if __name__ == "__main__":
    main()

Placed MARKET order for 200.0 shares of AAPL
Placed STOP order for 200.0 shares of AAPL at $150.0
OpenOrder. ID: 55, Symbol: AAPL, Action: SELL, Quantity: 200
OrderStatus. ID: 55, Status: PreSubmitted, Filled: 0, Remaining: 200, AvgFillPrice: 0.0
OpenOrder. ID: 54, Symbol: AAPL, Action: BUY, Quantity: 200
OrderStatus. ID: 54, Status: Submitted, Filled: 0, Remaining: 200, AvgFillPrice: 0.0
